In [19]:
import duckdb
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd

# df = pd.read_parquet('datasets/competencia_01_aum.parquet')


# tabla = pa.Table.from_pandas(df)

# # write df as a parquet dataset
# ds.write_dataset(tabla, base_dir = "datasets/competencia_01_aumpy",
#                   format="parquet",
#                  partitioning= ["foto_mes"],
#                  max_rows_per_file = 42000, max_rows_per_group = 42000,
#                  existing_data_behavior = "overwrite_or_ignore")

In [21]:

dataset = ds.dataset('datasets/competencia_01_aum', format="parquet")

In [31]:
columnas = dataset.schema.names

In [35]:
# get all columns that does not contain the pattern "lag2|variacion2"
cols_selected  = [col for col in columnas if not pd.Series(col).str.contains("lag2|variacion2", regex=True).any()]

In [36]:
dataset.to_table(columns=cols_selected).to_pandas().head()

,numero_de_cliente,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,...,variacion1_decil_Visa_madelantopesos,variacion1_decil_Master_madelantodolares,variacion1_decil_Visa_madelantodolares,variacion1_decil_cliente_edad,variacion1_total_mlimitecompra,variacion1_total_mfinanciacion_limite,variacion1_total_msaldototal,variacion1_total_patrimonio,variacion1_total_payroll,variacion1_total_prestamos
0,1.456243e+09,1.0,0.0,2.0,23.0,16.0,337.82,-10320.42,0.00,284.12,...,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,-1.797693e+308,0.000000e+00,0.0
1,1.456253e+09,1.0,0.0,0.0,36.0,17.0,179.79,-3781.47,-204.81,0.00,...,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,0.000000e+00,0.000000e+00,0.000000e+00,1.797693e+308,1.797693e+308,0.0
2,1.456585e+09,1.0,0.0,0.0,41.0,18.0,184.74,4255.09,9.47,0.00,...,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,0.000000e+00,1.797693e+308,1.797693e+308,0.0
3,1.456605e+09,1.0,0.0,0.0,23.0,18.0,3389.51,29675.85,7.81,0.00,...,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,0.000000e+00,0.000000e+00,0.000000e+00,1.797693e+308,0.000000e+00,0.0
4,1.456652e+09,1.0,0.0,0.0,36.0,18.0,1363.43,341968.88,65.19,666.63,...,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,1.797693e+308,0.000000e+00,0.0
